In [15]:
# Descargar los pesos preentrenados de YOLOv3-Tiny
!wget https://pjreddie.com/media/files/yolov3-tiny.weights -O yolov3-tiny.weights

# Clonar el repositorio de YOLOv3
!git clone https://github.com/AlexeyAB/darknet.git

# Cambiar al directorio del repositorio clonado
import os
os.chdir('darknet')

# Compilar darknet
!make


--2024-08-25 20:49:25--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35434956 (34M) [application/octet-stream]
Saving to: ‘yolov3-tiny.weights’

yolov3-tiny.weights 100%[===================>]  33.79M  12.3MB/s    in 2.8s    

2024-08-25 20:49:28 (12.3 MB/s) - ‘yolov3-tiny.weights’ saved [35434956/35434956]

fatal: destination path 'darknet' already exists and is not an empty directory.


NotADirectoryError: [Errno 20] Not a directory: 'darknet'

In [ ]:
# Celda 2b: Creación del archivo de configuración de datos para YOLOv3
import os
import glob
import shutil

# Directorios de imágenes
train_dir = '/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/Train-Images'
val_dir = '/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/Val-Images'

# Obtener todas las rutas de las imágenes
train_images = glob.glob(os.path.join(train_dir, '*.jpg'))
val_images = glob.glob(os.path.join(val_dir, '*.jpg'))

# Guardar las rutas en archivos .txt
with open('/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/Train-Images/train.txt', 'w') as f:
    for path in train_images:
        f.write(f"{path}\n")

with open('/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/Val-Images/val.txt', 'w') as f:
    for path in val_images:
        f.write(f"{path}\n")

print("Archivos train.txt y val.txt creados con éxito.")

# Crear el archivo yolov3.names
names_file_content = """botellas de plástico
botellas de vidrio
frascos
latas
cartón
detergentes"""

with open('/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/yolov3.names', 'w') as f:
    f.write(names_file_content)

print("Archivo yolov3.names creado con éxito.")

# Crear la carpeta backup
backup_dir = '/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/backup'
os.makedirs(backup_dir, exist_ok=True)
print(f"Carpeta backup creada en {backup_dir}.")

# Crear el archivo .data para YOLOv3
data_file_content = f"""
classes= 6
train  = /workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/Train-Images/train.txt
valid  = /workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/Val-Images/val.txt
names  = /workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/yolov3.names
backup = /workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/backup/
"""

# Guardar el archivo como yolov3-tiny.data
current_path = '/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/yolov3-tiny.data'
with open(current_path, 'w') as f:
    f.write(data_file_content)
print("Archivo yolov3-tiny.data guardado con éxito")

In [ ]:
# Celda 2b: Mover el archivo yolov3-tiny.data
# Ruta de destino en darknet/cfg
destination_dir = '/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/darknet/cfg'
destination_path = os.path.join(destination_dir, 'yolov3-tiny.data')

# Crear el directorio cfg si no existe
os.makedirs(destination_dir, exist_ok=True)

# Mover el archivo
shutil.move(current_path, destination_path)
print(f"Archivo movido a '{destination_path}'.")

# Verificar si el archivo fue movido correctamente
if os.path.exists(destination_path):
    print(f"El archivo '{destination_path}' existe.")
else:
    print(f"El archivo '{destination_path}' no existe.")

In [ ]:
# Celda 4b: Entrenamiento del modelo YOLOv3
# Descargar los pesos preentrenados
!wget https://pjreddie.com/media/files/darknet53.conv.74 -O darknet53.conv.74

# Entrenar el modelo YOLOv3
!./darknet detector train /workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/darknet/cfg/yolov3-tiny.data /workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/darknet/cfg/yolov3-tiny.cfg /workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/yolov3-tiny.weights -clear

In [12]:
import os

# Verificar si el archivo existe en la ubicación correcta
libdarknet_path = '/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/darknet/libdarknet.so'

if os.path.exists(libdarknet_path):
    print(f"Archivo {libdarknet_path} encontrado correctamente.")
else:
    print(f"Archivo {libdarknet_path} no encontrado. Verifica la compilación de Darknet.")


Archivo /workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/darknet/libdarknet.so encontrado correctamente.


In [16]:
#Celda 5b: Evaluación del Rendimiento de Yolov3
import cv2 as cv
import numpy as np
import os

# Cargar la red YOLOv3 usando OpenCV DNN
cfg_path = '/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/darknet/cfg/yolov3-tiny.cfg'
weights_path = '/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/backup/yolov3-tiny_final.weights'
names_path = '/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/yolov3.names'

net = cv.dnn.readNetFromDarknet(cfg_path, weights_path)
net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv.dnn.DNN_TARGET_CPU)

# Cargar las clases desde el archivo .names
with open(names_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Función para realizar la detección
def detect_image(image_path):
    image = cv.imread(image_path)
    blob = cv.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_outputs = net.forward(net.getUnconnectedOutLayersNames())

    boxes = []
    confidences = []
    class_ids = []

    h, w = image.shape[:2]
    
    for output in layer_outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                box = detection[0:4] * np.array([w, h, w, h])
                (centerX, centerY, width, height) = box.astype("int")

                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            color = [int(c) for c in np.random.uniform(0, 255, size=(3,))]
            cv.rectangle(image, (x, y), (x + w, y + h), color, 2)
            text = f"{classes[class_ids[i]]}: {confidences[i]:.4f}"
            cv.putText(image, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return image

# Evaluar una imagen
image_path = '/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/Val-Images/sample.jpg'
output_image = detect_image(image_path)

# Mostrar el resultado
cv.imshow('Detections', output_image)
cv.waitKey(0)
cv.destroyAllWindows()


[ WARN:0@963.868] global loadsave.cpp:248 findDecoder imread_('/workspaces/TFM-Master4-0/TFM-Master4-0/Codigo_Reduccion_Dataset/Val-Images/sample.jpg'): can't open/read file: check file path/integrity
[ WARN:0@963.868] global dnn_utils.cpp:166 blobFromImagesWithParamsImpl Red/blue color swapping requires at least three image channels.


error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'
